In [1]:
%load_ext autoreload
%autoreload 2

# Add the project directory to our module path
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Import the rest of our modules
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

%matplotlib inline

# Import py files
from src import cm_class_LRM as LRM # Logistic Regression Model Class
from src import cm_class_KNN as KNN # K-Nearest Neighbor Model Class
from src import cm_functions_balancing as cmbal
from src import cm_functions_preprocessing as cmpre
from src import cm_functions_tuning as cmtune

# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Classifier Libraries
from sklearn.linear_model import LogisticRegression, RidgeCV, LassoCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier

# Other Libraries
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, cross_val_score
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import make_scorer, precision_score
from collections import Counter
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.utils import resample

# Create paths to folders in repo
data_folder = '../../data/'
src_folder = '../../src/'

In [3]:
df=pd.read_csv('../../data/initial_clean_lc', index_col=[0])
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,KS,128,415,False,True,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,False,True,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,False,False,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,True,False,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,True,False,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
# create the dummy matrix
st_matrix = ohe.fit_transform(df[['state']])
# make it a pandas df
df_states = pd.DataFrame(st_matrix)
# set the column labels
df_states.columns= list(ohe.categories_[0])
# join the dataframes and drop the original column
df_new = df.join(df_states)
df_new.drop('state', axis=1, inplace=True)

In [6]:
df_new

,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,128,415,False,True,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,107,415,False,True,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,137,415,False,False,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84,408,True,False,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75,415,True,False,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,False,True,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3329,68,415,False,False,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3330,28,510,False,False,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3331,184,510,True,False,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X, X2, X3, y= cmpre.prepare_dataset(df_new)

# Model 1, Using X2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, random_state=42, test_size = .25)

In [9]:
# create kfolds object
kf = KFold(n_splits = 5, shuffle=True, random_state = 42)

In [10]:
# create list to add recall scores
fold_count=0
for train_ind, val_ind in kf.split(X_train, y_train):
    fold_count+=1

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.ss_scale(X_t, X_val)

    # instantiate and fit SMOTE:
    smote = SMOTE(random_state = 42)
    X_t_sc_bal, y_t_bal = smote.fit_resample(X_t_sc, y_t)
    
    clf = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        random_state = 15)

    train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, estimator=clf, scoring=recall,cv=10)

    # Predict
    y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

    conf_matrix = confusion_matrix(y_val,y_pred)

    # Store results
    performance.loc['GBC_exp_lr0.01_msl0.1_mss0.136_ss',
                    ['Train_Recall','Test_Recall','Test_Specificity']] = [
        train_cv.mean(),
        recall_score(y_val,y_pred),
        conf_matrix[0,0]/conf_matrix[0,:].sum()
    ]
    print(performance)

                                  Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.01_msl0.1_mss0.136_ss     0.853018        0.85         0.857143
                                  Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.01_msl0.1_mss0.136_ss     0.844363    0.853333         0.884706
                                  Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.01_msl0.1_mss0.136_ss     0.841455    0.838235          0.87037
                                  Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.01_msl0.1_mss0.136_ss     0.851554         0.9         0.867442
                                  Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.01_msl0.1_mss0.136_ss     0.853512    0.876923          0.87788


In [11]:
X_train_ss, X_test_ss = cmpre.ss_scale(X_train, X_test)

smote = SMOTE(random_state = 15)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ss, y_train)
    
gb = GradientBoostingClassifier(loss = 'exponential', learning_rate = 0.01, 
                                min_samples_leaf = 0.1, min_samples_split = 0.13636363636363638,
                                random_state = 15)
gb.fit(X_train_resampled, y_train_resampled)

print(f'Train: {recall_score(y_train, gb.predict(X_train_ss))}')
print(f'Test:  {recall_score(y_test, gb.predict(X_test_ss))}')

Train: 0.8631284916201117
Test:  0.888


In [12]:
X_train_ss, X_test_ss = cmpre.ss_scale(X_train, X_test)

smote = SMOTE(random_state = 15)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ss, y_train)

param_grid = {
    "loss":["exponential"],
    "learning_rate": [0.01],
    "min_samples_split": [0.1],
    "min_samples_leaf": [0.13636363636363638],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse",  "mae"],
    }

clf = GradientBoostingClassifier(random_state=15)

search = GridSearchCV(estimator=clf,param_grid=param_grid,n_jobs=-1,scoring='recall')

# Train search object
search.fit(X_train_resampled, y_train_resampled)

# Get best estimator
best = search.best_estimator_
print('Best parameters: \n\n',search.best_params_,'\n')

Best parameters: 

 {'criterion': 'mae', 'learning_rate': 0.01, 'loss': 'exponential', 'max_depth': 3, 'min_samples_leaf': 0.13636363636363638, 'min_samples_split': 0.1} 



# Model 2, Using X2

- StandardScaler()
- {'criterion': 'mae', 'learning_rate': 0.01, 'loss': 'exponential', 'max_depth': 3, 'min_samples_leaf': 0.13636363636363638, 'min_samples_split': 0.1} 

In [14]:
fold_count=0
for train_ind, val_ind in kf.split(X_train, y_train):
    fold_count+=1

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.ss_scale(X_t, X_val)

    # instantiate and fit SMOTE:
    smote = SMOTE(random_state = 42)
    X_t_sc_bal, y_t_bal = smote.fit_resample(X_t_sc, y_t)
    
    clf = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)

    train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, estimator=clf, scoring=recall,cv=10)

    # Predict
    y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

    conf_matrix = confusion_matrix(y_val,y_pred)

    # Store results
    performance.loc['GBC_exp_maxd3_mae_ss',
                    ['Train_Recall','Test_Recall','Test_Specificity']] = [
        train_cv.mean(),
        recall_score(y_val,y_pred),
        conf_matrix[0,0]/conf_matrix[0,:].sum()
    ]
    print(performance)

                     Train_Recall Test_Recall Test_Specificity
GBC_exp_maxd3_mae_ss     0.858237        0.85         0.857143
                     Train_Recall Test_Recall Test_Specificity
GBC_exp_maxd3_mae_ss     0.849612    0.853333         0.884706
                     Train_Recall Test_Recall Test_Specificity
GBC_exp_maxd3_mae_ss     0.849646    0.838235          0.87037
                     Train_Recall Test_Recall Test_Specificity
GBC_exp_maxd3_mae_ss     0.857987         0.9         0.869767
                     Train_Recall Test_Recall Test_Specificity
GBC_exp_maxd3_mae_ss     0.853512    0.876923          0.87788


In [15]:
X_train_ss, X_test_ss = cmpre.ss_scale(X_train, X_test)

smote = SMOTE(random_state = 15)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ss, y_train)
    
gb = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)
gb.fit(X_train_resampled, y_train_resampled)

print(f'Train: {recall_score(y_train, gb.predict(X_train_ss))}')
print(f'Test:  {recall_score(y_test, gb.predict(X_test_ss))}')

Train: 0.8631284916201117
Test:  0.888


# MODEL 3: ADASYN Balancing

In [34]:
fold_count=0
for train_ind, val_ind in kf.split(X_train, y_train):
    fold_count+=1

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.ss_scale(X_t, X_val)

    # instantiate and fit ADASYN: Oversample minority
    X_t_sc_bal, y_t_bal = ADASYN(sampling_strategy='minority',random_state=42).fit_resample(X_t_sc, y_t)
    
    
    clf = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        random_state = 15)

    train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, estimator=clf, scoring=recall,cv=10)

    # Predict
    y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

    conf_matrix = confusion_matrix(y_val,y_pred)

    # Store results
    performance.loc['GBC_ADASYN',
                    ['Train_Recall','Test_Recall','Test_Specificity']] = [
        train_cv.mean(),
        recall_score(y_val,y_pred),
        conf_matrix[0,0]/conf_matrix[0,:].sum()
    ]
    print(performance)

           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN     0.832246        0.85         0.857143
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN      0.84551    0.853333         0.884706
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN     0.832084    0.838235          0.87037
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN     0.832845         0.9         0.860465
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN      0.82503    0.876923          0.87788
